In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
import h5py
from imageio import imread
import numpy as np
import os
from os.path import join
import train_interactions
import pandas as pd
from interactions_results import toarray

In [ ]:
DATA_DIR = '/home/matej/prace/ferda/data/interactions'
EXPERIMENT_DIR = '/home/matej/prace/ferda/experiments/171129_1957'

NAMES = 'ant1_x, ant1_y, ant1_major, ant1_minor, ant1_angle, ' \
        'ant2_x, ant2_y, ant2_major, ant2_minor, ant2_angle'
FORMATS = 10 * 'f,'

pd.set_option('precision', 2)

In [ ]:
with h5py.File(DATA_DIR + '/results_inter_test.h5', 'r') as hf:
    y_test = np.core.records.fromarrays(hf['data'][:].transpose(), names=NAMES, formats=FORMATS)
    y_test_df = pd.DataFrame(y_test)

with h5py.File(EXPERIMENT_DIR + '/predictions.h5', 'r') as hf:
    pred = np.core.records.fromarrays(hf['data'][:].transpose(), names=NAMES, formats=FORMATS)
    pred_df = pd.DataFrame(pred)

# Predictions

In [ ]:
pred_df.describe()

In [ ]:
_ = plt.hist2d(pred['ant1_x'], pred['ant1_y'], bins=40, range=((0, 199), (0, 199)))

In [ ]:
_ = plt.hist2d(pred['ant2_x'], pred['ant2_y'], bins=40, range=((0, 199), (0, 199)))

In [ ]:
pred_df['ant1_angle'].hist()

In [ ]:
(pred_df['ant1_angle'] % 180).hist()

In [ ]:
pred_df['ant2_angle'].hist()

In [ ]:
(pred_df['ant2_angle'] % 180).hist()

# Prediction Errors

In [ ]:
xy1_error = np.linalg.norm(toarray(y_test[['ant1_x', 'ant1_y']]) - toarray(pred[['ant1_x', 'ant1_y']]), 2, axis=1)
xy2_error = np.linalg.norm(toarray(y_test[['ant2_x', 'ant2_y']]) - toarray(pred[['ant2_x', 'ant2_y']]), 2, axis=1)
xy1_error_df = pd.DataFrame(xy1_error)
xy2_error_df = pd.DataFrame(xy2_error)

In [ ]:
angle1_error = train_interactions.angle_absolute_error(toarray(y_test)[:, :5], toarray(pred)[:, :5], np)
angle2_error = train_interactions.angle_absolute_error(toarray(y_test)[:, 5:], toarray(pred)[:, 5:], np)
angle1_error_df = pd.DataFrame(angle1_error)
angle2_error_df = pd.DataFrame(angle2_error)

In [ ]:
df = pd.DataFrame.from_items([('xy (px)', [np.concatenate((xy1_error, xy2_error)).mean()]), 
                         ('angle (deg)', np.concatenate((angle1_error, angle2_error)).mean()),
                         ('xy1 (px)', xy1_error.mean()),
                         ('xy2 (px)', xy2_error.mean()),
                         ('angle1 (px)', angle1_error.mean()),
                         ('angle2 (px)', angle2_error.mean())])
df.style.set_caption('MAE')
df

In [ ]:
xy1_error_df.hist()

In [ ]:
xy2_error_df.hist()

In [ ]:
angle1_error_df.hist()

In [ ]:
angle2_error_df.hist()

In [ ]:
min_error = 20
(pd.DataFrame(pred['ant1_angle'][angle1_error.flatten() > min_error]) % 180).hist()
plt.title('Histogram of test angles1 where the error exceeds %d deg.' % min_error)

In [ ]:
min_error = 20
(pd.DataFrame(pred['ant2_angle'][angle2_error.flatten() > min_error]) % 180).hist()
plt.title('Histogram of test angles2 where the error exceeds %d deg.' % min_error)

# Model

In [ ]:
from IPython.display import SVG
from keras.utils.vis_utils import model_to_dot

model = train_interactions.model()
SVG(model_to_dot(model, show_shapes=True).create(prog='dot', format='svg'))